In [1]:
import numpy as np
from math import e
from scipy import linalg as LA
from scipy import integrate
from sklearn.metrics import mean_squared_error

In [2]:
def linear_system(t,Y,params):
    #param    the array containing all the entries of a matrix
    #this is a function that describes a linear system dX=AX, where A is n by n, X is n by 1
    c11=params[0]
    c12=params[1]
    c13=params[2]
    c21=params[3]
    c22=params[4]
    c23=params[5]
    c31=params[6]
    c32=params[7]
    c33=params[8]
    x=Y[0]
    y=Y[1]
    z=Y[2]
    dx=c11*x+c12*y+c13*z
    dy=c21*x+c22*y+c23*z
    dz=c31*x+c32*y+c33*z
    return([dx,dy,dz])

In [3]:
def num_solve(func,tspan,y0,params):
    sol=integrate.solve_ivp(func,tspan,y0,args=(params,))
    sol=sol.y
    Y=[]
    x=sol[0]
    x=x[-1]
    y=sol[1]
    y=y[-1]
    z=sol[2]
    z=z[-1]
    Y.append(x)
    Y.append(y)
    Y.append(z)
    return Y

In [4]:
tspan=[0,10]
y0=[2,0,-1]
params=(2,0,-2,0,4,0,-2,0,5)
numerical_solution=num_solve(linear_system,tspan,y0,params)#the numerical solution using RK45
print(numerical_solution)

[9.054661286697442e+25, 0.0, -1.8109322573394884e+26]


In [5]:
def matrix(params):
    #turning the array as a matrix
    params=np.array(params)
    params=np.reshape(params,(3,3))
    return params

In [6]:
params=(2,0,-2,0,4,0,-2,0,5)
A=matrix(params)

In [7]:
def cons_generator(matrix,Y):
    #Y      initial conditions
    w,v=LA.eigh(matrix)
    C=np.linalg.solve(v,Y)
    return C

In [8]:
Y=[2,0,-1]
cons_params=cons_generator(A,Y)
print(cons_params)

[ 1.34164079  0.         -1.78885438]


In [9]:
def analy_solve(t,matrix,cons_params):
    #solving a linear system dX=AX using the most general solution
    #cons_params    containing n constants for n terms in the general solution
    X=[]
    w,v=LA.eigh(matrix)
    L=len(w)
    for i in range(L):
        e_term=cons_params[i]*(e**(w[i]*t))
        y=np.dot(e_term,v[:,i])
        X.append(y)
        Y=np.sum(X,axis=0)
    return Y
    #the function eventually returns the explicit solution

In [10]:
explicit_solution=analy_solve(10,A,cons_params)
print(explicit_solution)

[ 9.13605912e+25  0.00000000e+00 -1.82721182e+26]


In [11]:
MSE=mean_squared_error(explicit_solution,numerical_solution)
print('the numerical solution is',numerical_solution,', the actual solution is',explicit_solution,', the mean square error is',MSE,'.' )

the numerical solution is [9.054661286697442e+25, 0.0, -1.8109322573394884e+26] , the actual solution is [ 9.13605912e+25  0.00000000e+00 -1.82721182e+26] , the mean square error is 1.1042678377166507e+48 .
